# 1. polglot-ko-1.3b를 squarelike/sharegpt_deepl_ko_translation로 파인 튜닝한 모델

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "aeolian83/Gugugo_for_DnD_v0.6"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")

In [2]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [3]:
from transformers import StoppingCriteria, StoppingCriteriaList

class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, stops = [], encounters=1):
        super().__init__()
        self.stops = [stop for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if torch.all((stop == input_ids[0][-len(stop):])).item():
                return True

        return False

stop_words = ["</끝>"]
stop_words_ids = [tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in stop_words]
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_words_ids)])

In [4]:
def gen(lan="en", x=""):
    if (lan == "ko"):
        prompt = f"### 한국어: {x}</끝>\n### 영어:"
    else:
        prompt = f"### 영어: {x}</끝>\n### 한국어:"
    gened = model.generate(
        **tokenizer(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=2048,
        temperature=0.001,
        no_repeat_ngram_size=10,
        early_stopping=True,
        eos_token_id=2,
        stopping_criteria=stopping_criteria
    )
    return tokenizer.decode(gened[0]).replace(prompt+" ", "")

In [5]:
gen(lan="en", x="The Steward informed me that a mysterious package has somehow arrived on my ship. I should speak with her to learn more.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/aeolian83/anaconda3/envs/translateDnD/lib/python3.10/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


'관리자는 나에게 신비한 물건이 내 배에 도착했다고 말했소. 그녀와 이야기를 해봐야겠소.</끝>'

In [6]:
gen(lan="en", x="I've found myself in some sort of tournament, replete with spectators. I'm told that &lt;b&gt;Humaire&lt;/b&gt; is the custodian of this place, and may be able to provide answers regarding the strange invitation. She is currently &lt;b&gt;topside,&lt;/b&gt; watching the events.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'나는 일종의 토너먼트에 참가하게 되었고, 구경꾼들과 함께 있게 되었다. 나는 &lt;b&gt;호박의 관리인이며, 이상한 초대에 대한 답을 제공할 수 있을 것이다. 그녀는 현재 &lt;b&gt;의 중앙에 있고, 그 장소를 내려다보고 있다.</끝>'

In [7]:
gen(lan="en", x="&lt;b&gt;Humaire&lt;/b&gt; told me to speak with her in the &lt;b&gt;Hall of Memories&lt;/b&gt; on the &lt;b&gt;west side of the temple&lt;/b&gt; beneath the arena.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'&lt;b&gt>Humaire&lt;/B&gt; told me to see her in the &lt;b–t–Hall of Memories&lte–> /b–g–west of the temple&lt–</b–g–with the arena</끝>'

In [8]:
gen(lan="en", x="I've found myself in some sort of tournament, replete with spectators. I'm told that <b>Humaire</b> is the custodian of this place, and may be able to provide answers regarding the strange invitation. She is currently <b>topside,</b> watching the events.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/aeolian83/anaconda3/envs/translateDnD/lib/python3.10/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


'나는 일종의 토너먼트에 참가하게 되었고, 구경꾼들과 함께 있게 되었다. 나는 <b>Humaire</B>가 이 장소의 수호자라고 들었고, 이상한 초대에 대한 답을 제공할 수 있을 것이다. 그녀는 현재 <b>의 위치에 있다.</끝>'

In [9]:
gen(lan="en", x="<b>Humaire</b> told me to speak with her in the <b>Hall of Memories</b> on the <b>west side of the temple</b> beneath the arena.")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/home/aeolian83/anaconda3/envs/translateDnD/lib/python3.10/site-packages/transformers/generation/utils.py:1468: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


'<b>휴머레이트</b>가 <b>기억의 전당</b>의 <b>서쪽에 있는 <b>사원에서 그녀와 이야기하라고 했소.</끝>'